<a href="https://colab.research.google.com/github/kairamilanifitria/Bootcamp-NLP/blob/main/Project%202%20Text%20Summarization/2_Dev%2BTest_BERT%2BSummarize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Load the dataset
dev_df = pd.read_csv('/content/drive/MyDrive/Bootcamp AI/Dataset/Task2 : Text Summarization/fine-tune_dataset/dev_df.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Bootcamp AI/Dataset/Task2 : Text Summarization/fine-tune_dataset/test_df.csv')

# Basic overview
print(dev_df.shape)
print(dev_df.info())
print(dev_df.isnull().sum())

print(test_df.shape)
print(test_df.info())
print(test_df.isnull().sum())

(9540, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9540 entries, 0 to 9539
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   9540 non-null   int64 
 1   url                  9540 non-null   object
 2   text                 9540 non-null   object
 3   abstractive_summary  9540 non-null   object
 4   extractive_summary   9540 non-null   object
dtypes: int64(1), object(4)
memory usage: 372.8+ KB
None
id                     0
url                    0
text                   0
abstractive_summary    0
extractive_summary     0
dtype: int64
(9775, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9775 entries, 0 to 9774
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   9775 non-null   int64 
 1   url                  9775 non-null   object
 2   text                 9775 non

In [2]:
# prompt: delete the column "id" and "url" in those dataset

dev_df = dev_df.drop(['id', 'url'], axis=1)
test_df = test_df.drop(['id', 'url'], axis=1)


In [3]:
test_df.iloc[1]

,1
text,liputan6 com yogyakarta datang kereta api dari...
abstractive_summary,berangkat kereta api dari jalur selatan kembal...
extractive_summary,demikian ungkap orang tugas pt kereta api indo...


In [4]:
!pip install datasets transformers torch accelerate

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.2 MB/s eta 0:00:00


# Set Tokenizer

In [5]:
import pandas as pd
import torch
from datasets import load_dataset, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, DataCollatorForSeq2Seq, EncoderDecoderModel
from transformers import pipeline

In [6]:
# Check if GPU is available and set device accordingly
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Using device: cuda


In [7]:
# Load pre-trained BERT model for text summarization (abstractive and extractive)
model_name = "cahya/bert2bert-indonesian-summarization"

tokenizer = BertTokenizer.from_pretrained("cahya/bert2bert-indonesian-summarization")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token

model = EncoderDecoderModel.from_pretrained("cahya/bert2bert-indonesian-summarization").to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/230k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.09k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/999M [00:00<?, ?B/s]

In [10]:
# prompt: make a code to training model text summarization using the data in dev_df

from transformers import TrainingArguments, Trainer

# Convert pandas DataFrame to Hugging Face Dataset
dev_dataset = Dataset.from_pandas(dev_df)

# Convert pandas DataFrame to Hugging Face Dataset
dev_dataset = Dataset.from_pandas(dev_df)

# Define a function to preprocess the data
def preprocess_function(examples):
    # Use the model's maximum sequence length instead of 1024
    model_inputs = tokenizer(
        examples["text"], max_length=512, truncation=True, padding="max_length"
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["abstractive_summary"], max_length=128, truncation=True, padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing to the dataset
tokenized_dev_datasets = dev_dataset.map(preprocess_function, batched=True)



# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=3,
    push_to_hub=False,
    fp16=True,
)

# Define data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dev_datasets,
    eval_dataset=tokenized_dev_datasets,
    data_collator=data_collator,
)

# Train the model
trainer.train()


Map:   0%|          | 0/9540 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:643: FutureWarning:

Epoch,Training Loss,Validation Loss
1,0.597700,0.445598
2,0.474600,0.363401
3,0.401300,0.332887


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 40, 'min_length': 20, 'early_stopping': True, 'num_beams': 10, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:643: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
Some

TrainOutput(global_step=3579, training_loss=0.5263195799928565, metrics={'train_runtime': 2641.5928, 'train_samples_per_second': 10.834, 'train_steps_per_second': 1.355, 'total_flos': 1.755718630244352e+16, 'train_loss': 0.5263195799928565, 'epoch': 3.0})

In [12]:
# prompt: save the model that has already trained

trainer.save_model("./trained_model")


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 40, 'min_length': 20, 'early_stopping': True, 'num_beams': 10, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}


In [14]:
# prompt: save the trained model in my local desktop

from google.colab import files

!zip -r trained_model.zip ./trained_model

files.download('trained_model.zip')


  adding: trained_model/ (stored 0%)
  adding: trained_model/config.json (deflated 80%)
  adding: trained_model/model.safetensors (deflated 7%)
  adding: trained_model/training_args.bin (deflated 51%)
  adding: trained_model/generation_config.json (deflated 41%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [30]:
import os

# Define the path in Google Drive where you want to save the model
output_dir = '/content/drive/MyDrive/Bootcamp AI/Dataset/Task2 : Text Summarization/model_abstractive'

# Create the directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the trained model and tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Model saved to {output_dir}")


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 40, 'min_length': 20, 'early_stopping': True, 'num_beams': 10, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}


Model saved to /content/drive/MyDrive/Bootcamp AI/Dataset/Task2 : Text Summarization/model_abstractive


In [31]:
!ls '/content/drive/MyDrive/Bootcamp AI/Dataset/Task2 : Text Summarization/model_abstractive'

config.json		model.safetensors	 tokenizer_config.json
generation_config.json	special_tokens_map.json  vocab.txt


In [33]:
from transformers import BertForSequenceClassification, BertTokenizer

# Define the correct Google Drive path
model_dir = '/content/drive/MyDrive/Bootcamp AI/Dataset/Task2 : Text Summarization/model_abstractive'

# Load the model and tokenizer from the local directory
model = BertForSequenceClassification.from_pretrained(model_dir)
tokenizer = BertTokenizer.from_pretrained(model_dir)


You are using a model of type encoder-decoder to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/Bootcamp AI/Dataset/Task2 : Text Summarization/model_abstractive and are newly initialized: ['classifier.bias', 'classifier.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.

In [39]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

def generate_summaries(text):
    # For abstractive summarization, load a pre-trained model like T5/BART
    # Use AutoModelForSeq2SeqLM to load a model suitable for text generation
    # Instead of model_name, use a suitable pre-trained model for summarization
    abstractive_summarizer = pipeline("summarization", model="t5-small", tokenizer="t5-small")
    abstractive_summary = abstractive_summarizer(text, truncation=True, max_length=100)[0]['summary_text']

    # For extractive summarization, continue using BertForSequenceClassification
    # Assuming you have loaded 'model' and 'tokenizer' for this purpose previously
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model(**inputs)
    extractive_summary = " ".join([tokenizer.decode(idx, skip_special_tokens=True) for idx in torch.argmax(outputs.logits, dim=-1)])

    return abstractive_summary, extractive_summary

In [40]:
# Example of how to use the model for prediction
example_text = test_df['text'][0]  # Replace with the text you want to summarize
abstractive_summary, extractive_summary = generate_summaries(example_text)

print("Abstractive Summary:", abstractive_summary)
print("Extractive Summary:", extractive_summary)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Abstractive Summary: liputan6 com jakarta jaksa agung resmi cekal tommy alias ibrahim kali ini kait dengan milik senjata api amunisi dan bahan ledak di apartemen cemara dan rumah di jalan alam segar iii no 23 pondok inda
Extractive Summary: [ U N K ]


In [42]:
test_df.head(2)

,text,abstractive_summary,extractive_summary
0,liputan6 com jakarta jaksa agung resmi cekal h...,tommy soeharto kembali cekal atas kasus milik ...,cekal tommy alias ibrahim kali ini kait dengan...
1,liputan6 com yogyakarta datang kereta api dari...,berangkat kereta api dari jalur selatan kembal...,demikian ungkap orang tugas pt kereta api indo...


In [44]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=8099627df5490f4cb1743031db1408979f94384fdf7683a9b5d7810b6ea201cd
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [45]:
from rouge_score import rouge_scorer

def calculate_rouge(reference_text, candidate_text):
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
  scores = scorer.score(reference_text, candidate_text)
  return scores

# Example usage:
reference_summary = "tommy soeharto kembali cekal atas kasus milik senjata api amunisi dan bahan ledak cekal kepada tommy lama tahun itu laku sejak 19 oktober 2001"
candidate_summary = "liputan6 com jakarta jaksa agung resmi cekal tommy alias ibrahim kali ini kait dengan milik senjata api amunisi dan bahan ledak di apartemen cemara dan rumah di jalan alam segar iii no 23 pondok inda"


rouge_scores = calculate_rouge(reference_summary, candidate_summary)

print("ROUGE Score Abstractive:")
print(f"ROUGE-1: {rouge_scores['rouge1'].fmeasure:.4f}")
print(f"ROUGE-2: {rouge_scores['rouge2'].fmeasure:.4f}")
print(f"ROUGE-L: {rouge_scores['rougeL'].fmeasure:.4f}")


ROUGE Score Abstractive:
ROUGE-1: 0.3051
ROUGE-2: 0.2105
ROUGE-L: 0.2712


In [46]:
from rouge_score import rouge_scorer

def calculate_rouge(reference_text, candidate_text):
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
  scores = scorer.score(reference_text, candidate_text)
  return scores

# Example usage:
reference_summary = "cekal tommy alias ibrahim kali ini kait dengan milik senjata api amunisi dan bahan ledak di apartemen cemara dan rumah di jalan alam segar iii no moeljohardjo jelas cekal kepada tommy lama tahun itu laku sejak 19 oktober 2001"
candidate_summary = "liputan6 com jakarta jaksa agung resmi cekal tommy alias ibrahim kali ini kait dengan milik senjata api amunisi dan bahan ledak di apartemen cemara dan rumah di jalan alam segar iii no 23 pondok inda"


rouge_scores = calculate_rouge(reference_summary, candidate_summary)

print("ROUGE Score Abstractive:")
print(f"ROUGE-1: {rouge_scores['rouge1'].fmeasure:.4f}")
print(f"ROUGE-2: {rouge_scores['rouge2'].fmeasure:.4f}")
print(f"ROUGE-L: {rouge_scores['rougeL'].fmeasure:.4f}")


ROUGE Score Abstractive:
ROUGE-1: 0.7027
ROUGE-2: 0.6944
ROUGE-L: 0.7027
